In [4]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem.snowball import FrenchStemmer
from tqdm import tqdm  # Barre de progression

# Téléchargement de la ressource 'punkt' de NLTK.  Nécessaire une seule fois.
# nltk.download('punkt')

# === Nettoyage et Prétraitement du Texte ===

def nettoyer_et_tronquer(texte, limite=2000):  # Increased limit.  Experiment!
    """
    Nettoie et tronque un texte.

    Args:
        texte (str): Le texte à nettoyer.
        limite (int): Le nombre maximum de caractères à conserver.

    Returns:
        str: Le texte nettoyé et tronqué.
    """
    texte_propre = texte.strip().replace('\n', ' ')  # Supprime les espaces et sauts de ligne en trop
    return texte_propre[:limite]  # Tronque le texte

# Utilisation du stemming (racinisation) pour le français
stemmer = FrenchStemmer()

def normaliser_texte(texte):
    """
    Normalise un texte : minuscule, suppression de caractères spéciaux, racinisation.

    Args:
        texte (str): Le texte à normaliser.

    Returns:
        str: Le texte normalisé.
    """
    texte = texte.lower()  # Convertit en minuscules
    texte = re.sub(r"[^a-zàâçéèêëîïôûùüÿñæœ\s-]", "", texte)  # Supprime les caractères non-alphanumériques
    mots = nltk.word_tokenize(texte, language='french')  # Tokenisation (découpage en mots)
    mots_racines = [stemmer.stem(mot) for mot in mots]  # Racinisation des mots
    return " ".join(mots_racines)  # Reconstitue le texte à partir des racines

# === Chargement des Données ===
print("Chargement des données...")
df_fiches = pd.read_csv("/Users/arsenegery/Desktop/Finance_Poste_DS/Données_&_Notebook/info_particulier_impot.csv")

# Concaténation du titre et du texte, puis nettoyage
df_fiches["Texte_Concat"] = (
    df_fiches["Titre"].fillna("") + " : " + df_fiches["Texte"].fillna("")
).apply(nettoyer_et_tronquer)

# === Indexation Thématique (Texte Complet) ===
# Crée un index inversé : mot -> liste des indices des documents où le mot apparaît.
index_theme = defaultdict(set)
for idx, texte_concat in enumerate(df_fiches["Texte_Concat"]):
    for mot in normaliser_texte(texte_concat).split():  # Normalisation + indexation sur le texte complet
        index_theme[mot].add(idx)


# === Chargement du Modèle SBERT (Sentence-BERT) ===
print("🔄 Chargement du modèle SBERT...")
modele = SentenceTransformer("dangvantuan/sentence-camembert-large") # Modèle pour le français
fiches_textes = df_fiches["Texte_Concat"].tolist()
fiches_embeddings = modele.encode(fiches_textes, convert_to_tensor=True).cpu().numpy() # Crée les embeddings (représentations vectorielles) des textes des fiches.

# === Moteur de Recherche ===
def chercher_fiche(question, top_n=5, seuil_similarite=0.35):  # Added threshold
    """
    Recherche les fiches les plus pertinentes pour une question donnée.

    Args:
        question (str): La question posée par l'utilisateur.
        top_n (int): Le nombre maximum de résultats à retourner.
        seuil_similarite (float): Le seuil de similarité cosinus minimale pour considérer un résultat comme pertinent.

    Returns:
        list: Une liste de dictionnaires, chacun représentant un résultat de recherche.
              Chaque dictionnaire contient : 'question', 'titre', 'url', 'similarité', 'nb_fiches_consultées', 'extrait'.
    """
    mots_question = set(normaliser_texte(question).split()) # Normalise la question (minuscule, racinisation)
    indices_possibles = set()
    for mot in mots_question:
        indices_possibles.update(index_theme.get(mot, [])) # Récupère les indices des documents contenant au moins un mot de la question

    if not indices_possibles:  # Si aucun mot de la question n'est trouvé dans l'index
        indices_possibles = set(range(len(fiches_textes)))

    sous_ensemble = [fiches_embeddings[i] for i in indices_possibles]
    question_embedding = modele.encode(question, convert_to_tensor=True).cpu().numpy().reshape(1, -1)  # Crée l'embedding de la question
    similarites = cosine_similarity(question_embedding, sous_ensemble).flatten() # Calcule la similarité cosinus entre la question et les documents

    # Get top_n indices (using argsort for efficiency)
    top_n_indices_local = np.argsort(similarites)[::-1][:top_n]
    top_n_indices_global = [list(indices_possibles)[i] for i in top_n_indices_local]

    resultats = []
    for i in range(min(top_n, len(top_n_indices_global))):  # Gère le cas où il y a moins de top_n résultats
        score_similarite = float(similarites[top_n_indices_local[i]])

        # Extrait un passage pertinent du texte (implémentation simple)
        question_normalisee = normaliser_texte(question)
        mots_question = question_normalisee.split()
        texte_concat = df_fiches["Texte_Concat"].iloc[top_n_indices_global[i]]
        texte_normalise = normaliser_texte(texte_concat) # Texte normalisé

        meilleur_extrait = ""
        max_mots_correspondants = 0

        # This improvement takes O(n*m) time, where n : length of the text, and m: length of the question
        mots_texte = texte_normalise.split()
        for j in range(0, len(mots_texte) - 15):
          extrait = " ".join(mots_texte[j:j+15])
          mots_correspondants = 0
          for word in mots_question:
            if word in extrait:
                mots_correspondants+=1
          if mots_correspondants > max_mots_correspondants:
            meilleur_extrait = extrait
            max_mots_correspondants = mots_correspondants


        resultats.append({
            "question": question,
            "titre": df_fiches["Titre"].iloc[top_n_indices_global[i]],
            "url": df_fiches["URL"].iloc[top_n_indices_global[i]],
            "similarité": score_similarite,
            "nb_fiches_consultées": len(indices_possibles),
            "extrait": meilleur_extrait + "...",  # Ajoute l'extrait
        })

    # Filtre les résultats en dessous du seuil de similarité
    resultats_filtres = [r for r in resultats if r['similarité'] >= seuil_similarite]

    if not resultats_filtres:
        return []  # Retourne une liste vide si aucun résultat n'est pertinent

    return resultats_filtres


# === Évaluation ===
df_questions = pd.read_csv("/Users/arsenegery/Desktop/Finance_Poste_DS/Données_&_Notebook/questions_fiches_fip.csv")

def evaluer_chatbot(df_questions, top_n=5):
    """
    Évalue les performances du chatbot.

    Args:
        df_questions (pd.DataFrame):  DataFrame contenant les questions et les réponses attendues.
        top_n (int): Le nombre de résultats à considérer pour le calcul de l'accuracy.
    """
    correct_at_1 = 0  # Compteur de réponses correctes en 1ère position
    correct_at_n = 0  # Compteur de réponses correctes dans les top_n résultats
    reciprocal_ranks = []  # Liste des reciprocal ranks (1/rang de la bonne réponse)
    zero_results = 0

    for _, row in tqdm(df_questions.iterrows(), total=len(df_questions), desc="Évaluation"):  # tqdm pour la barre de progression
        question = row["question"]
        correct_num_texte = row["num_texte"] # L'indice de la bonne réponse

        resultats = chercher_fiche(question, top_n=top_n)

        if len(resultats)==0:
          zero_results +=1

        # Find the correct answer
        trouve = False
        for rang, resultat in enumerate(resultats):
            # Find the index using 'num_texte' (more reliable)
            retrieved_index = df_fiches[df_fiches["URL"] == resultat["url"]].index.tolist()
            if len(retrieved_index) > 0:
                retrieved_index = retrieved_index[0]
            else:
                retrieved_index = -1
            if retrieved_index == correct_num_texte: # Comparaison des index
                if rang == 0:
                    correct_at_1 += 1  # Incrémente si la réponse est en première position
                correct_at_n += 1  # Incrémente si la réponse est dans les top_n
                reciprocal_ranks.append(1 / (rang + 1))  # Ajoute le reciprocal rank
                trouve = True
                break  # Arrête la recherche une fois que la réponse est trouvée

        if not trouve:
            reciprocal_ranks.append(0)



    accuracy_at_1 = correct_at_1 / len(df_questions)  # Calcule l'accuracy@1
    accuracy_at_n = correct_at_n / len(df_questions)  # Calcule l'accuracy@n
    mrr = np.mean(reciprocal_ranks)  # Calcule le Mean Reciprocal Rank (MRR)
    print(f"Précision@1: {accuracy_at_1:.4f}")  # Affiche l'accuracy@1
    print(f"Précision@{top_n}: {accuracy_at_n:.4f}")  # Affiche l'accuracy@n
    print(f"Rang Réciproque Moyen (MRR): {mrr:.4f}")  # Affiche le MRR
    print(f"Nombre de questions sans aucun résultat: {zero_results}") # Affiche le nombre de questions sans aucun résultat

# === Interface Console ===
if __name__ == "__main__":
    print("\n💬 Chatbot DGFiP (version locale)")
    print("Evaluation des performances du chatbot...")
    evaluer_chatbot(df_questions, top_n=5)  # Garder top_n = 5
    while True:
        q = input("\nPosez votre question (ou 'exit') :\n> ")
        if q.lower() == "exit":
            break
        resultats = chercher_fiche(q, seuil_similarite=0.35)

        if resultats:
            if resultats[0]['similarité'] < 0.35:
                print("\n😞 Je ne suis pas sûr d'avoir une réponse parfaite, mais ces documents pourraient être liés :")
            else:
                print(f"\n🔎 {len(resultats)} fiches correspondantes trouvées :\n")

            for i, rep in enumerate(resultats):
                print(f"--- Résultat {i+1} ---")
                print(f"📌 Fiche : {rep['titre']}")
                print(f"🔗 URL  : {rep['url']}")
                print(f"📊 Score : {rep['similarité']:.4f} sur {rep['nb_fiches_consultées']} fiches analysées")
                print(f"💬 Extrait: {rep['extrait']}...")  # Use 'extrait', not 'snippet'
        else:
            print("😞 Aucune fiche trouvée...\n")

Chargement des données...
🔄 Chargement du modèle SBERT...


No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with MEAN pooling.



💬 Chatbot DGFiP (version locale)
Evaluation des performances du chatbot...


Évaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1427/1427 [04:22<00:00,  5.44it/s]


Précision@1: 0.5620
Précision@5: 0.8360
Rang Réciproque Moyen (MRR): 0.6692
Nombre de questions sans aucun résultat: 6


...


Posez votre question (ou 'exit') :
>  Quel est le but fondamental d'un contrat d'assurance-vie ?



🔎 3 fiches correspondantes trouvées :

--- Résultat 1 ---
📌 Fiche : L'ASSURANCE-VIE ET LE PEA
🔗 URL  : https://www.impots.gouv.fr/particulier/lassurance-vie-et-le-pea-0
📊 Score : 0.4637 sur 113 fiches analysées
💬 Extrait: contrat dépargn et dassur sign entre un assur et un assureur dont le but est......
--- Résultat 2 ---
📌 Fiche : ÉPARGNE RETRAITE
🔗 URL  : https://www.impots.gouv.fr/particulier/epargne-retraite
📊 Score : 0.3798 sur 113 fiches analysées
💬 Extrait: industriel et commercial ou bénéfic non commercial le différent produit éligibl le perp ce contrat......
--- Résultat 3 ---
📌 Fiche : LES REVENUS MOBILIERS
🔗 URL  : https://www.impots.gouv.fr/particulier/les-revenus-mobiliers
📊 Score : 0.3727 sur 113 fiches analysées
💬 Extrait: valeur mobili que vous possed action part de sarl oblig bon de capitalis contrat dassurance-v......



Posez votre question (ou 'exit') :
>  A quoi sert une assurance-vie ?



🔎 1 fiches correspondantes trouvées :

--- Résultat 1 ---
📌 Fiche : L'ASSURANCE-VIE ET LE PEA
🔗 URL  : https://www.impots.gouv.fr/particulier/lassurance-vie-et-le-pea-0
📊 Score : 0.3734 sur 97 fiches analysées
💬 Extrait: lassurance-v et le pe lassurance-v et le pe lassurance-v définit lassurance-v est un contrat dépargn......



Posez votre question (ou 'exit') :
>  exit
